In [16]:
# Load Transaction Data from a CSV File

### DISCLAIMER: 
### FOR ALL TAX-RELATED QUESTIONS AND CONCERNS PLEASE ASK A CERTIFIED ACCOUNTANT

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# TODO: move your MINT.com transactions.csv into this project directory
# / the directory where you are running jupyter notebook
PATH_TO_YOUR_TRANSACTIONS = "transactions.csv"
PATH_TO_RENTS = "rents.csv"

# Set the name of the column where the transactions are categorized
# In Mint this is usually "Category".  I use Mint's category to link 
# transactions to a property and the add a seperate "Label" column
# after exporting the data for a property from Min
CATEGORY = "Label"


# Load the transaction data from the csv into a dataframe
parse_dates = ['Date']
df = pd.read_csv(PATH_TO_YOUR_TRANSACTIONS, parse_dates=parse_dates)
df.set_index(['Date'], inplace=True)
df['Amount'] = df['Amount'].astype(float)

# Get rid of Mint columns that we don't care about
df.drop(['Original Description','Actual Amount','Notes', 'Category', 'Account Name'], axis=1, inplace=True)
# Ignore everything other than the security related transactions
df = df[(df[CATEGORY] == 'Security') | (df[CATEGORY] == 'Security-Income')]
df.sort_index(inplace=True)
df.head()

,Description,Amount,Transaction Type,Label,Unit
Date,,,,,
2016-12-27,Purchased Liability on Alberto/Paula 1E Deposit,675.0,credit,Security,1E
2016-12-27,Purchased Liability on Apt 1W Security Deposit,700.0,credit,Security,1W
2016-12-27,Purchased Liability on Apt 2F Security Deposit,1100.0,credit,Security,2F
2017-04-13,Check 604 - Return Security Deposit Alberto/Pa...,700.0,debit,Security,1E
2017-05-22,122 Spring Street 1E Tosin Security Deposit,700.0,credit,Security,1E


In [18]:
## Show the Security Related Transactions for Each Unit

# Read in the rents data so we know the units
rents = pd.read_csv(PATH_TO_RENTS)
rents.set_index(['Year'], inplace=True)

for unit in rents.columns:
    unit_df = df[df['Unit'] == unit]
    unit_df


,Description,Amount,Transaction Type,Label,Unit
Date,,,,,
2016-12-27,Purchased Liability on Apt 2F Security Deposit,1100.0,credit,Security,2F
2017-08-03,Check 1151 ($1000) - 122 Spring St Upstairs Se...,1100.0,debit,Security,2F
2017-09-08,122 Spring Street Upstairs Security Deposit - ...,1100.0,credit,Security,2F
2018-10-15,122 Spring Street Upstairs Security Deposit - ...,1100.0,credit,Security-Income,2F
2018-11-01,122 Spring Street Upstairs Deposit Rachel,575.0,credit,Security,2F
2018-11-14,122 Spring Street Upstairs Matt Security Deposit,600.0,credit,Security,2F


,Description,Amount,Transaction Type,Label,Unit
Date,,,,,
2016-12-27,Purchased Liability on Apt 1W Security Deposit,700.0,credit,Security,1W
2018-09-28,Alex Marissa Security Refund - cash via ATM,700.0,debit,Security,1W
2018-10-25,122 Spring 1W security deposit part 1,700.0,credit,Security,1W
2018-10-25,122 Spring 1W security deposit part 2,100.0,credit,Security,1W


,Description,Amount,Transaction Type,Label,Unit
Date,,,,,
2016-12-27,Purchased Liability on Alberto/Paula 1E Deposit,675.0,credit,Security,1E
2017-04-13,Check 604 - Return Security Deposit Alberto/Pa...,700.0,debit,Security,1E
2017-05-22,122 Spring Street 1E Tosin Security Deposit,700.0,credit,Security,1E
2018-07-03,Google Wallet Tosin Deposit Refund,700.0,debit,Security,1E
2018-07-11,122 Spring St Apt 1E Marisha and Sid Deposit,700.0,credit,Security,1E
